In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf

In [ ]:
df = pd.read_csv('weight_encoded_2.csv')
df.sample(5)

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
98,0,22.000000,1.500000,49.000000,1,0,2.000000,1.000000,2,0,2.000000,0,3.000000,0.000000,3,4,1
1111,0,33.864257,1.679299,77.355417,1,1,2.768020,2.137068,2,0,1.873591,0,0.966973,0.691944,3,0,3
1043,0,19.090023,1.562434,70.442775,1,0,2.748243,3.070386,2,0,1.962322,0,1.145752,1.000000,3,3,3
704,0,37.218161,1.593894,63.320629,1,1,2.374640,3.000000,2,0,2.000000,0,2.892922,0.480813,2,0,2
513,1,18.000000,1.840138,60.000000,1,1,3.000000,4.000000,2,0,2.000000,0,2.000000,0.000000,3,0,0


In [ ]:
X = df.drop('NObeyesdad', axis=1)
y = df['NObeyesdad']

In [ ]:
categorical_cols = ['Gender', 'family_history_with_overweight', 'FAVC', 'CAEC', 'SMOKE', 'SCC', 'MTRANS']
preprocessor = ColumnTransformer(
    transformers=[('encoder', OneHotEncoder(), categorical_cols)],
    remainder='passthrough'
)

In [ ]:
X = preprocessor.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2, random_state=42
)

In [ ]:
# Scale the features using StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
tf.random.set_seed(42)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(np.unique(y)), activation='softmax')
])

In [ ]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(lr=0.001),
    metrics=['accuracy']
)

In [ ]:
# Train the model
history = model.fit(X_train_scaled, y_train, epochs=100, validation_data=(X_test_scaled, y_test))

Epoch 1/100
28/28 [==============================] - 3s 21ms/step - loss: 1.0892 - accuracy: 0.5841 - val_loss: 0.7959 - val_accuracy: 0.6771
Epoch 2/100
28/28 [==============================] - 0s 9ms/step - loss: 0.6148 - accuracy: 0.7668 - val_loss: 0.5368 - val_accuracy: 0.7578
Epoch 3/100
28/28 [==============================] - 0s 9ms/step - loss: 0.4056 - accuracy: 0.8543 - val_loss: 0.4099 - val_accuracy: 0.8565
Epoch 4/100
28/28 [==============================] - 0s 9ms/step - loss: 0.2843 - accuracy: 0.9013 - val_loss: 0.3756 - val_accuracy: 0.8610
Epoch 5/100
28/28 [==============================] - 0s 12ms/step - loss: 0.1974 - accuracy: 0.9406 - val_loss: 0.3106 - val_accuracy: 0.9148
Epoch 6/100
28/28 [==============================] - 0s 6ms/step - loss: 0.1279 - accuracy: 0.9675 - val_loss: 0.2844 - val_accuracy: 0.8969
Epoch 7/100
28/28 [==============================] - 0s 9ms/step - loss: 0.0802 - accuracy: 0.9888 - val_loss: 0.2668 - val_accuracy: 0.9013
Epoch 8/100

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test)

7/7 [==============================] - 0s 3ms/step - loss: 0.4238 - accuracy: 0.9283


In [ ]:
# Print the evaluation results
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

Test Loss: 0.4238133728504181
Test Accuracy: 0.9282511472702026


In [ ]:
df = pd.read_csv('filtered_dataset.csv')
df.sample(5)

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
266,Male,20.000000,1.800000,75.000000,no,yes,3.000000,4.000000,Always,no,3.000000,no,2.000000,0.000000,Sometimes,Public_Transportation,Normal_Weight
491,Female,17.082867,1.640824,43.365005,no,yes,2.815157,3.000000,Sometimes,no,2.911187,no,2.595128,1.380204,Sometimes,Public_Transportation,Insufficient_Weight
153,Male,21.000000,1.620000,70.000000,no,yes,2.000000,1.000000,no,no,3.000000,no,1.000000,0.000000,Sometimes,Public_Transportation,Overweight_Level_I
315,Male,19.000000,1.800000,80.000000,no,yes,2.000000,1.000000,Sometimes,no,2.000000,no,2.000000,1.000000,Sometimes,Public_Transportation,Normal_Weight
827,Male,26.703710,1.802871,85.770013,yes,yes,2.872121,3.051804,Sometimes,no,2.111913,no,0.843709,0.153559,Sometimes,Automobile,Overweight_Level_I
